In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
from google.colab import runtime
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pylab as plt
from IPython.display import Image
from google.colab import files
from pyparsing import Word, hexnums, WordEnd, Optional, alphas, alphanums
import cv2
import os
import struct
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow import keras
from keras.optimizers import Adamax
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras.applications.xception import preprocess_input
from keras.applications import Xception
from sklearn.model_selection import train_test_split
import pickle
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Flatten, Dense
from keras.models import Model
from keras.applications import EfficientNetV2S
from keras.applications.efficientnet_v2 import preprocess_input as preprocess_input2

In [4]:
def create_model(ishape,lr):

	model = tf.keras.applications.Xception(
          weights= None,
          input_shape = ishape,
          classes=1,
          classifier_activation='sigmoid'
          )

	model.compile(loss='binary_crossentropy',
                    optimizer=Adamax(learning_rate=lr),
                    metrics=['accuracy', 'Precision', 'Recall']
                    )

	return model

In [5]:
def create_model2(ishape,lr):

	model = tf.keras.applications.EfficientNetV2S(
          weights= None,
          input_shape = ishape,
          classes=1,
          classifier_activation='sigmoid'
          )

	model.compile(loss='binary_crossentropy',
                    optimizer=Adamax(learning_rate=lr),
                    metrics=['accuracy', 'Precision', 'Recall']
                    )

	return model

In [9]:
ishape = (128,256,3)
model = create_model(ishape, 0.001)
idg = ImageDataGenerator(preprocessing_function=tf.keras.applications.xception.preprocess_input)
training_data = pd.read_csv("/content/drive/MyDrive/HHDataset/HHFolds/fold1train.csv")
test_data = pd.read_csv("/content/drive/MyDrive/HHDataset/HHFolds/fold1test.csv")
training_data[['GR']] = training_data[['GR']].astype(str)
test_data[['GR']] = test_data[['GR']].astype(str)
train_set = idg.flow_from_dataframe(training_data, directory="/content/drive/MyDrive/HHDataset/HHFolds/HeaderPlusImages/",
                                    x_col="ImagePath", y_col="GR",
                                    class_mode="binary", shuffle=False, target_size=ishape[:2], batch_size=32)
test_set  = idg.flow_from_dataframe(test_data, directory="/content/drive/MyDrive/HHDataset/HHFolds/HeaderPlusImages/",
                                    x_col="ImagePath", y_col="GR",
                                    class_mode="binary", shuffle=False, target_size=ishape[:2], batch_size=32)
with tf.device('/device:GPU:0'):
  model.fit(train_set, epochs=45, batch_size=32, verbose=1)
with tf.device('/device:GPU:0'):
  results = model.evaluate(test_set, batch_size=32, verbose=1)
with tf.device('/device:GPU:0'):
  results2 = model.predict(test_set, batch_size=1, verbose=1)
# print("=====misclassifications=====")
# for i in range(len(results2)):
#   if (results2[i] >= 0.5 and test_data.iloc[i]["GR"] == '0') or (results2[i] < 0.5 and test_data.iloc[i]["GR"] == '1'):
#     print(results2[i], test_data.iloc[i]["ImagePath"])

Found 899 validated image filenames belonging to 2 classes.
Found 100 validated image filenames belonging to 2 classes.
Epoch 1/45
29/29 [==============================] - 29s 298ms/step - loss: 0.3461 - accuracy: 0.8509 - precision: 0.8248 - recall: 0.8940
Epoch 2/45
29/29 [==============================] - 9s 293ms/step - loss: 0.1518 - accuracy: 0.9455 - precision: 0.9391 - recall: 0.9536
Epoch 3/45
29/29 [==============================] - 9s 301ms/step - loss: 0.0681 - accuracy: 0.9766 - precision: 0.9675 - recall: 0.9868
Epoch 4/45
29/29 [==============================] - 9s 297ms/step - loss: 0.0251 - accuracy: 0.9889 - precision: 0.9847 - recall: 0.9934
Epoch 5/45
29/29 [==============================] - 9s 307ms/step - loss: 0.0273 - accuracy: 0.9933 - precision: 0.9912 - recall: 0.9956
Epoch 6/45
29/29 [==============================] - 9s 304ms/step - loss: 0.0059 - accuracy: 0.9989 - precision: 1.0000 - recall: 0.9978
Epoch 7/45
29/29 [==============================] - 9s 30

In [11]:
ishape = (128,256,3)
model = create_model(ishape, 0.001)
idg = ImageDataGenerator(preprocessing_function=tf.keras.applications.xception.preprocess_input)
training_data = pd.read_csv("/content/drive/MyDrive/HHDataset/HHFolds/fold1train.csv")
test_data = pd.read_csv("/content/drive/MyDrive/HHDataset/HHFolds/fold1test.csv")
training_data[['GR']] = training_data[['GR']].astype(str)
test_data[['GR']] = test_data[['GR']].astype(str)
train_set = idg.flow_from_dataframe(training_data, directory="/content/drive/MyDrive/HHDataset/HHFolds/HeaderPlusImages/",
                                    x_col="ImagePath", y_col="GR",
                                    class_mode="binary", shuffle=False, target_size=ishape[:2], batch_size=32)
test_set  = idg.flow_from_dataframe(test_data, directory="/content/drive/MyDrive/HHDataset/HHFolds/HeaderPlusImages/",
                                    x_col="ImagePath", y_col="GR",
                                    class_mode="binary", shuffle=False, target_size=ishape[:2], batch_size=32)
with tf.device('/device:GPU:0'):
  model.fit(train_set, epochs=15, batch_size=32, verbose=1)
with tf.device('/device:GPU:0'):
  results = model.evaluate(test_set, batch_size=32, verbose=1)
with tf.device('/device:GPU:0'):
  results2 = model.predict(test_set, batch_size=1, verbose=1)
# print("=====misclassifications=====")
# for i in range(len(results2)):
#   if (results2[i] >= 0.5 and test_data.iloc[i]["GR"] == '0') or (results2[i] < 0.5 and test_data.iloc[i]["GR"] == '1'):
#     print(results2[i], test_data.iloc[i]["ImagePath"])

Found 899 validated image filenames belonging to 2 classes.
Found 100 validated image filenames belonging to 2 classes.
Epoch 1/15
29/29 [==============================] - 30s 291ms/step - loss: 0.3682 - accuracy: 0.8331 - precision: 0.8086 - recall: 0.8764
Epoch 2/15
29/29 [==============================] - 9s 295ms/step - loss: 0.1530 - accuracy: 0.9433 - precision: 0.9351 - recall: 0.9536
Epoch 3/15
29/29 [==============================] - 9s 313ms/step - loss: 0.0698 - accuracy: 0.9822 - precision: 0.9740 - recall: 0.9912
Epoch 4/15
29/29 [==============================] - 9s 310ms/step - loss: 0.0476 - accuracy: 0.9867 - precision: 0.9783 - recall: 0.9956
Epoch 5/15
29/29 [==============================] - 9s 305ms/step - loss: 0.0106 - accuracy: 0.9956 - precision: 0.9978 - recall: 0.9934
Epoch 6/15
29/29 [==============================] - 9s 304ms/step - loss: 0.0278 - accuracy: 0.9933 - precision: 0.9891 - recall: 0.9978
Epoch 7/15
29/29 [==============================] - 9s 29